In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from scipy.fft import fft, ifft, fftfreq
from matplotlib.pyplot import figure
import math as m
from segment import Segment
from func import *

In [ ]:
NUM_PORTS = 4
SIZE = 100
imp0 = 377
xs0 = 50  # source position for the segment 0
tau = 0.3
# kappa = 1j* m.sqrt(1-tau**2) # not working yet for complex number

maxTime = 5000

rings_no = 10
N_seg = rings_no*2

# sources = [[-1,-1],[0,2],[3,5],[2,0],[5,3],[2,4]]
# couplings = [[tau,kappa],[tau,kappa],[tau,kappa],[tau,kappa],[tau,kappa],[tau,kappa]]

sources = Sources(rings_no)  #working for any number of rings
couplings = Couplings(rings_no, tau)

print(sources)
# print(couplings)
ez_tab = np.zeros([N_seg, maxTime, SIZE], dtype=complex)

In [ ]:
for qTime in range(maxTime):
    s = cosMod(qTime, maxTime, complex_signal=False, f0=3e15) 
    plt.plot(range(maxTime)),s)
    plt.show()  
    


In [ ]:
f_array = []

s_tab = np.zeros(maxTime, dtype=complex)

for ii in range(N_seg):
    f = Segment(SIZE)
    f_array.append(f)
# print(f_array)

for qTime in range(30):
    s = cosMod(qTime, maxTime, complex_signal=False, f0=3e15) 
    s_tab[qTime] = s
    
    for ii in range(N_seg):
        s1 = sources[ii][0]
        s2 = sources[ii][1]
        tau_ = couplings[ii][0]
        kappa_ = couplings[ii][1]
        
        xs = xs0
        if (sources[ii][0] == -1):
            s_new = s   
            xs = xs0
        else:
            s_new = tau_*f_array[s1].ez[-1] + kappa_*f_array[s2].ez[-1]
            xs = 2  
            
        # print(s_new)
        
        f_array[ii].hy_update()
        f_array[ii].hy_sources(s_new,xs)
        f_array[ii].ez_update()
        f_array[ii].ez_sources(s_new,xs)
        

        ez_tab[ii,qTime,:] = f_array[ii].ez[:]   

for ii in range(N_seg):
    filename = f"data/ez_tab_{ii:04d}.csv"  # e.g., ez_tab_0000.txt, ez_tab_0001.txt, ...
    np.savetxt(filename, ez_tab[ii, :, :], delimiter=",")

In [ ]:
for qTime in range(9):
    plt.plot(np.real(s_tab[qTime]))
plt.show()

In [ ]:
f, ax = plt.subplots(1,N_seg, figsize=(15,10)) 

vmax_val = np.max(np.real(ez_tab[0,:,:]))

for ii in range(N_seg):
    ax[ii].imshow(np.real(ez_tab[ii,:,:]), vmin = 0, vmax = vmax_val , cmap = 'Reds')
# plt.savefig("ez_real.png")


# plt.figure()
# plt.plot(s_tab)

# f, ax3 = plt.subplots(1,6, figsize=(10,15)) 
# for ii in range(6):
#     ax3[ii].imshow(abs(ez_tab[ii,:,:]), vmin = 0, vmax = vmax_val , cmap = 'Reds')
# plt.savefig("ez_abs.png")
# plt.plot(ez_tab[:,15])

In [ ]:
Ez = np.zeros((maxTime-1,SIZE), dtype=complex)
yf = np.zeros((maxTime-1,SIZE), dtype=complex)
N = maxTime-1
dt = 1e-17
xf = fftfreq(N,dt)

for ii in range(N_seg):
    filename = f"data/ez_tab_{ii:04d}.csv"
    df = pd.read_csv(filename)
    # print(df.shape)
    Ez[:,ii] = df.iloc[:,80]   #all rows of the 10th spatial point
    Ez[:,ii] -= np.mean(Ez[:,ii])
    
    signal = np.real(Ez[start:,ii])
    signal -= np.mean(signal)
    
    yf[:,ii] = abs(fft(Ez[:,ii]))
    
    #accessing particular columns - all elements(all time steps for a particular spatial coordinate.)
x = range(len(df.iloc[0,:])) 
# print(x)   

for ii in range(N_seg):
    figure(figsize=(8,8), dpi=80)
    plt.subplot(2,1,1)
    plt.plot(np.real(Ez[:,ii]), label={ii})
    plt.title("Temporal Evolution")
    plt.xlabel("time")
    plt.ylabel("Ez(t)")
    plt.legend()
    plt.subplot(2,1,2)
    plt.plot(xf,np.real(yf[:,ii]/max(yf[:,0])), label={ii})
    plt.xlim([0.,1e16])
    plt.title("Frequency Spectrum")
    plt.xlabel("frequency")
    plt.ylabel("Ez(f)")
    plt.legend()
    plt.show()

In [ ]:
from scipy.signal import windows
from numpy.fft import fft, fftfreq

start = 2000  # skip transient
signal = np.real(Ez[start:, ii])
signal -= np.mean(signal)  # remove DC

# Apply Hann window
win = windows.hann(len(signal))
signal_win = signal * win

# Zero-pad to make spectrum smoother
Npad = 2000
yf = np.abs(fft(signal_win, n=Npad))
xf = fftfreq(Npad, dt)

# Normalize & plot
plt.plot(xf, yf)
plt.xlim([0., 1e16])
plt.show()


In [ ]:
# Parameters

for tp in range(2000,2350,100):
    ez_tab_tp = ez_tab[:,tp,:]
    plot_field_ring(ez_tab_tp)

